In [1]:
import pandas as pd

class Contract:
    
    def __init__(self, wholesale_price, revenue_share, surplus_refund):
        self.wholesale_price = wholesale_price
        self.revenue_share = revenue_share
        self.surplus_refund = surplus_refund
        
        self.retail_price = 4250
        self.surplus_price = 2500
        self.factory_price = 2250
        
        self.demand_frame = pd.DataFrame({'Demand':range(6),
                                          'Probability':[.1, .15, .3, .2, .15, .1]})     
    
    def vendor_marginal_profit(self):
        return self.retail_price - self.wholesale_price - (1 - self.revenue_share)*self.retail_price
    
    def vendor_marginal_loss(self):
        return (self.wholesale_price - self.surplus_refund - self.surplus_price)
    
    def optimal_service(self):
        mp = self.vendor_marginal_profit()
        ml = self.vendor_marginal_loss()
        return mp/(mp+ml)
    
    def optimal_quantity(self):
        return round(self.optimal_service()*5, 0)
    
    def daily_view(self):
        quantity = self.optimal_quantity()
        self.demand_frame['Number Sold Apr-Aug'] = self.demand_frame['Demand'].clip(0, quantity)
        self.demand_frame['Surplus'] = (quantity - self.demand_frame['Demand']).clip(0, None)
        
        self.demand_frame['McCormick Revenue'] = self.demand_frame['Number Sold Apr-Aug']*self.retail_price\
                                                +self.demand_frame['Surplus']*self.surplus_price
        
        self.demand_frame['Surplus Refund'] = self.demand_frame['Surplus']*self.surplus_refund
        self.demand_frame['Revenue Sharing'] = self.demand_frame['McCormick Revenue']*(1 - self.revenue_share)
        
        self.demand_frame['McCormick Cost'] = quantity*self.wholesale_price
        
        self.demand_frame['McCormick Profit'] =  self.demand_frame['McCormick Revenue'] \
                                                -self.demand_frame['McCormick Cost'] \
                                                +self.demand_frame['Surplus Refund']\
                                                -self.demand_frame['Revenue Sharing']
        
        self.demand_frame['Wholesaler Revenue'] = quantity*self.wholesale_price      
        self.demand_frame['Wholesaler Cost'] = quantity*self.factory_price
        
        self.demand_frame['Wholesaler Profit'] =  self.demand_frame['Wholesaler Revenue'] \
                                                 -self.demand_frame['Wholesaler Cost'] \
                                                -self.demand_frame['Surplus Refund']\
                                                +self.demand_frame['Revenue Sharing']
        
        self.demand_frame['Supply Chain Revenue'] = self.demand_frame['McCormick Revenue']\
                                                   +self.demand_frame['Wholesaler Revenue']

        self.demand_frame['Supply Chain Cost'] = self.demand_frame['McCormick Cost']\
                                                +self.demand_frame['Wholesaler Cost']      
        
        self.demand_frame['Supply Chain Profit'] = self.demand_frame['McCormick Profit']\
                                                +self.demand_frame['Wholesaler Profit'] 
        
        return self.demand_frame  
    
    def vendor_expected_profit(self):
        df = self.daily_view()
        return (df['McCormick Profit']*df['Probability']).sum()
    
    def wholesale_expected_profit(self):
        df = self.daily_view()
        return (df['Wholesaler Profit']*df['Probability']).sum() 
    
    def supply_chain_expected_profit(self):
        df = self.daily_view()
        return ((df['Supply Chain Profit'])*df['Probability']).sum()

Question 1

Since we are able to always sell our lawnmowers higher than the factory price there is always a profit, meaning we should produce as many as possible. So we have:

In [2]:
max_supply = pd.DataFrame({'Demand':range(6),
                           'Probability':[.1, .15, .3, .2, .15, .1],
                           'Surplus':range(5, -1, -1)})   

In [3]:
max_supply['Surplus Profit'] = max_supply['Surplus']*250
max_supply['Regular Profit'] = max_supply['Demand']*2000
max_supply['Total Profit'] = max_supply['Surplus Profit'] + max_supply['Regular Profit']

In [4]:
max_supply

,Demand,Probability,Surplus,Surplus Profit,Regular Profit,Total Profit
0,0,0.10,5,1250,0,1250
1,1,0.15,4,1000,2000,3000
2,2,0.30,3,750,4000,4750
3,3,0.20,2,500,6000,6500
4,4,0.15,1,250,8000,8250
5,5,0.10,0,0,10000,10000


In [5]:
(max_supply['Total Profit']*max_supply['Probability']).sum()

5537.5

Question 2

In [6]:
traditional = Contract(wholesale_price = 3000,
                       revenue_share = 1,
                       surplus_refund = 0)

In [7]:
rev_share = Contract(wholesale_price = 2600,
                     revenue_share = .9,
                     surplus_refund = 0)

In [8]:
surplus_refund = Contract(wholesale_price = 3000,
                          revenue_share = 1,
                          surplus_refund = 400)

In [9]:
idx = ['Retail Price',
       'Wholesale Price',
       'Factory Price',
       'McCormick Revenue Share',
       'Surplus Refund',
       'McCormick MP',
       'McCormick ML',
       'McCormick Optimal Order Quantity',
       'McCormick Expected Profit',
       'Wholesaler Expected Profit',
       'Supply Chain Expected Profit']

In [10]:
traditional_series = pd.Series([traditional.retail_price,
                                traditional.wholesale_price,
                                traditional.factory_price,
                                traditional.revenue_share,
                                traditional.surplus_refund,
                                traditional.vendor_marginal_profit(),
                                traditional.vendor_marginal_loss(),
                                traditional.optimal_quantity(), 
                                traditional.vendor_expected_profit(), 
                                traditional.wholesale_expected_profit(), 
                                traditional.supply_chain_expected_profit()], 
                                
                                name = 'Traditional',
                                index = idx)

rev_share_series = pd.Series([rev_share.retail_price,
                              rev_share.wholesale_price,
                              rev_share.factory_price,
                              rev_share.revenue_share,
                              rev_share.surplus_refund,
                              rev_share.vendor_marginal_profit(),
                              rev_share.vendor_marginal_loss(),
                              rev_share.optimal_quantity(), 
                              rev_share.vendor_expected_profit(), 
                              rev_share.wholesale_expected_profit(), 
                              rev_share.supply_chain_expected_profit()],
                            
                              name = 'Revenue Sharing',
                              index = idx)

surplus_refund_series = pd.Series([surplus_refund.retail_price,
                                   surplus_refund.wholesale_price,
                                   surplus_refund.factory_price,
                                   surplus_refund.revenue_share,
                                   surplus_refund.surplus_refund,
                                   surplus_refund.vendor_marginal_profit(),
                                   surplus_refund.vendor_marginal_loss(),
                                   surplus_refund.optimal_quantity(), 
                                   surplus_refund.vendor_expected_profit(), 
                                   surplus_refund.wholesale_expected_profit(), 
                                   surplus_refund.supply_chain_expected_profit()], 
                                   
                                   name = 'Surplus-Refund',
                                   index = idx)

df_list = [traditional_series, rev_share_series, surplus_refund_series]

In [11]:
pd.concat(df_list, axis = 1)

,Traditional,Revenue Sharing,Surplus-Refund
Retail Price,4250.0,4250.00,4250.0
Wholesale Price,3000.0,2600.00,3000.0
Factory Price,2250.0,2250.00,2250.0
McCormick Revenue Share,1.0,0.90,1.0
Surplus Refund,0.0,0.00,400.0
McCormick MP,1250.0,1225.00,1250.0
McCormick ML,500.0,100.00,100.0
McCormick Optimal Order Quantity,4.0,5.00,5.0
McCormick Expected Profit,2112.5,2108.75,2807.5
Wholesaler Expected Profit,3000.0,3428.75,2730.0


In [12]:
traditional.daily_view()

,Demand,Probability,Number Sold Apr-Aug,Surplus,McCormick Revenue,Surplus Refund,Revenue Sharing,McCormick Cost,McCormick Profit,Wholesaler Revenue,Wholesaler Cost,Wholesaler Profit,Supply Chain Revenue,Supply Chain Cost,Supply Chain Profit
0,0,0.10,0,4.0,10000.0,0.0,0.0,12000.0,-2000.0,12000.0,9000.0,3000.0,22000.0,21000.0,1000.0
1,1,0.15,1,3.0,11750.0,0.0,0.0,12000.0,-250.0,12000.0,9000.0,3000.0,23750.0,21000.0,2750.0
2,2,0.30,2,2.0,13500.0,0.0,0.0,12000.0,1500.0,12000.0,9000.0,3000.0,25500.0,21000.0,4500.0
3,3,0.20,3,1.0,15250.0,0.0,0.0,12000.0,3250.0,12000.0,9000.0,3000.0,27250.0,21000.0,6250.0
4,4,0.15,4,0.0,17000.0,0.0,0.0,12000.0,5000.0,12000.0,9000.0,3000.0,29000.0,21000.0,8000.0
5,5,0.10,4,0.0,17000.0,0.0,0.0,12000.0,5000.0,12000.0,9000.0,3000.0,29000.0,21000.0,8000.0


In [13]:
rev_share.daily_view()

,Demand,Probability,Number Sold Apr-Aug,Surplus,McCormick Revenue,Surplus Refund,Revenue Sharing,McCormick Cost,McCormick Profit,Wholesaler Revenue,Wholesaler Cost,Wholesaler Profit,Supply Chain Revenue,Supply Chain Cost,Supply Chain Profit
0,0,0.10,0,5.0,12500.0,0.0,1250.0,13000.0,-1750.0,13000.0,11250.0,3000.0,25500.0,24250.0,1250.0
1,1,0.15,1,4.0,14250.0,0.0,1425.0,13000.0,-175.0,13000.0,11250.0,3175.0,27250.0,24250.0,3000.0
2,2,0.30,2,3.0,16000.0,0.0,1600.0,13000.0,1400.0,13000.0,11250.0,3350.0,29000.0,24250.0,4750.0
3,3,0.20,3,2.0,17750.0,0.0,1775.0,13000.0,2975.0,13000.0,11250.0,3525.0,30750.0,24250.0,6500.0
4,4,0.15,4,1.0,19500.0,0.0,1950.0,13000.0,4550.0,13000.0,11250.0,3700.0,32500.0,24250.0,8250.0
5,5,0.10,5,0.0,21250.0,0.0,2125.0,13000.0,6125.0,13000.0,11250.0,3875.0,34250.0,24250.0,10000.0


In [14]:
surplus_refund.daily_view()

,Demand,Probability,Number Sold Apr-Aug,Surplus,McCormick Revenue,Surplus Refund,Revenue Sharing,McCormick Cost,McCormick Profit,Wholesaler Revenue,Wholesaler Cost,Wholesaler Profit,Supply Chain Revenue,Supply Chain Cost,Supply Chain Profit
0,0,0.10,0,5.0,12500.0,2000.0,0.0,15000.0,-500.0,15000.0,11250.0,1750.0,27500.0,26250.0,1250.0
1,1,0.15,1,4.0,14250.0,1600.0,0.0,15000.0,850.0,15000.0,11250.0,2150.0,29250.0,26250.0,3000.0
2,2,0.30,2,3.0,16000.0,1200.0,0.0,15000.0,2200.0,15000.0,11250.0,2550.0,31000.0,26250.0,4750.0
3,3,0.20,3,2.0,17750.0,800.0,0.0,15000.0,3550.0,15000.0,11250.0,2950.0,32750.0,26250.0,6500.0
4,4,0.15,4,1.0,19500.0,400.0,0.0,15000.0,4900.0,15000.0,11250.0,3350.0,34500.0,26250.0,8250.0
5,5,0.10,5,0.0,21250.0,0.0,0.0,15000.0,6250.0,15000.0,11250.0,3750.0,36250.0,26250.0,10000.0
